In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from konlpy.tag import Okt

import os

from keybert import KeyBERT
import tqdm

c:\ProgramData\Anaconda3\envs\DINS\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [46]:
data = pd.read_csv('./Data/SBS.csv')

data.head(2)

,title,headline,date,link,content,category,site
0,"BTS ""웰컴 제네레이션…변화에 겁먹지 않고 새로운 길로""","미국 뉴욕을 찾은 방탄소년단, BTS가 ""지금 청년들은 변화에 겁먹기보단 '웰컴'이...",2021.09.20,https://news.sbs.co.kr//news/endPage.do?news_i...,"미국 뉴욕을 찾은 방탄소년단, BTS가 ""지금 청년들은 변화에 겁먹기보단 '웰컴...",국제,sbs
1,"고속도로 대부분 소통 원활…""추석 당일 정체 심할 것""",추석 전날인 오늘 저녁 전국 주요 고속도로의 정체가 대부분 해소돼 원활한 소통을 보...,2021.09.20,https://news.sbs.co.kr//news/endPage.do?news_i...,추석 전날인 오늘(20일) 저녁 전국 주요 고속도로의 정체가 대부분 해소돼 원활...,사회,sbs


In [45]:
# date 변경, link col_name 변경 및 저장

# data.rename(columns={'news_link' : 'link'}, inplace=True)

# data.date = data.date.str.split(' ').str[0]

# data.head(1)

# data.to_csv('./Data/SBS.csv', index=False)

In [26]:
# okt = Okt()

# contents_nouns = []

In [6]:
# contents_nouns.append(p.map(get_nouns, contents))

In [4]:
# for text in contents:
#     nouns = okt.nouns(text)
    
#     text_nouns = ''
#     for noun in nouns:
#         text_nouns += noun + ' '
    
#     contents_nouns.append(text_nouns)
#     print(text_nouns)

In [22]:
# contents = data.iloc[79573:].content

# len(contents)

252

In [47]:
data_sep = data.iloc[78237:]

len(data_sep)

1588

In [57]:
data_0913 = data_sep.groupby('date').get_group('2022.09.13').content
data_0914 = data_sep.groupby('date').get_group('2022.09.14').content
data_0915 = data_sep.groupby('date').get_group('2022.09.15').content
data_0916 = data_sep.groupby('date').get_group('2022.09.16').content
data_0917 = data_sep.groupby('date').get_group('2022.09.17').content
data_0918 = data_sep.groupby('date').get_group('2022.09.18').content
data_0919 = data_sep.groupby('date').get_group('2022.09.19').content

In [85]:
okt = Okt()

contents_nouns = []

In [86]:
dates = data_sep.date.unique()

In [87]:
datas = []

for date in dates:
    datas.append(data_sep.groupby('date').get_group(date).content)

In [95]:
contents_nouns = []

for contents in datas:
    content_nouns = ''
    
    for text in contents:
        text_nouns = okt.nouns(text)
    
        for noun in text_nouns:
            content_nouns += noun + ' '
            
    contents_nouns.append(content_nouns)

In [25]:
# content_nouns = ''

# for text in contents:
#     text_nouns = okt.nouns(text)
    
#     for noun in text_nouns:
#         content_nouns += noun + ' '
    

# contents_nouns.append(content_nouns)

In [102]:
model = KeyBERT('distilbert-base-nli-mean-tokens')

In [105]:
contents_keywords = []

for content_nouns in contents_nouns:
    contents_keywords.append(model.extract_keywords(content_nouns, keyphrase_ngram_range=(1,1), top_n=20, use_mmr=True))

In [131]:
result_keywords = []

date_idx = 0
for keywords in contents_keywords:
    date_keywords = []
    for keyword in keywords:
        date_keywords.append((dates[date_idx], ) + keyword) # (date, keyword, importance)
    
    date_idx += 1
    
    result_keywords.append(date_keywords)

In [138]:
result_keywords = np.array(result_keywords)

array([[['2022.09.13', '조선민주주의인민공화국', '0.8774'],
        ['2022.09.13', '미국배우조합상', '0.8682'],
        ['2022.09.13', '대륙간탄도미사일', '0.8636'],
        ['2022.09.13', '남아프리카공화국', '0.8603'],
        ['2022.09.13', '정보통신망법', '0.8461'],
        ['2022.09.13', '통신비밀보호법', '0.843'],
        ['2022.09.13', '콘세르트헤바우', '0.8408'],
        ['2022.09.13', '마이크로소프트', '0.8375'],
        ['2022.09.13', '칸국제영화제', '0.8359'],
        ['2022.09.13', '사법연수원', '0.8357'],
        ['2022.09.13', '아동복지법', '0.8324'],
        ['2022.09.13', '방탄소년단', '0.8311'],
        ['2022.09.13', '원삼국시대', '0.8247'],
        ['2022.09.13', '북아일랜드', '0.824'],
        ['2022.09.13', '서울대학교', '0.8232'],
        ['2022.09.13', '특별자치시', '0.8227'],
        ['2022.09.13', '지방자치단체', '0.8194'],
        ['2022.09.13', '로스앤젤레스', '0.813'],
        ['2022.09.13', '카타르월드컵', '0.8032'],
        ['2022.09.13', '신재생에너지', '0.8024']],

       [['2022.09.14', '전기통신사업법', '0.8768'],
        ['2022.09.14', '원자력발전소', '0.8664'],
        ['2022.09.14', '조직

In [143]:
result_keywords = result_keywords.reshape(140,3)

In [153]:
result = pd.DataFrame(result_keywords, columns=['date', 'keyword', 'importance'])

result.head(21)

,date,keyword,importance
0,2022.09.13,조선민주주의인민공화국,0.8774
1,2022.09.13,미국배우조합상,0.8682
2,2022.09.13,대륙간탄도미사일,0.8636
3,2022.09.13,남아프리카공화국,0.8603
4,2022.09.13,정보통신망법,0.8461
5,2022.09.13,통신비밀보호법,0.843
6,2022.09.13,콘세르트헤바우,0.8408
7,2022.09.13,마이크로소프트,0.8375
8,2022.09.13,칸국제영화제,0.8359
9,2022.09.13,사법연수원,0.8357


In [154]:
result.to_csv('./Data/keywords.csv', index=False)

In [114]:
import pymysql

In [155]:
keywords = pd.read_csv('./Data/keywords.csv')

keywords.head(5)

,date,keyword,importance
0,2022.09.13,조선민주주의인민공화국,0.8774
1,2022.09.13,미국배우조합상,0.8682
2,2022.09.13,대륙간탄도미사일,0.8636
3,2022.09.13,남아프리카공화국,0.8603
4,2022.09.13,정보통신망법,0.8461


In [156]:
sql_insert = 'INSERT INTO keywords (date, keyword, importance) VALUES (%s, %s, %s)'

In [157]:
conn = pymysql.connect(host='localhost', user='root', password='1234', db='dins', charset='utf8')
cursor = conn.cursor()


for index, row in keywords.iterrows():
    cursor.execute(sql_insert, (row.date, row.keyword, row.importance))

conn.commit()
conn.close()

In [106]:
# keywords = model.extract_keywords(contents_nouns, keyphrase_ngram_range=(1,1), top_n=15, use_mmr=True)

# keywords